## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-22-13-05-22 +0000,nypost,US chess superstar Daniel Naroditsky was incoh...,https://nypost.com/2025/10/22/us-news/chess-su...
1,2025-10-22-13-02-00 +0000,wsj,Treasury Yields Extend Fall as Markets Wait fo...,https://www.wsj.com/articles/jgbs-edge-lower-a...
2,2025-10-22-13-00-49 +0000,nypost,Queens mom arrested for leaving newborn at NYC...,https://nypost.com/2025/10/22/us-news/queens-m...
3,2025-10-22-13-00-00 +0000,missionlocal,View from the water taxi,https://missionlocal.org/2025/10/sf-photos-vie...
4,2025-10-22-12-59-29 +0000,nyt,Russia Hits Kindergarten and Power Plants Hour...,https://www.nytimes.com/2025/10/22/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
37,trump,54
65,israel,15
67,gaza,13
77,house,12
145,new,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
63,2025-10-22-10-00-00 +0000,nypost,Sen. John Kennedy talks Trump’s social media h...,https://nypost.com/2025/10/22/us-news/sen-john...,104
304,2025-10-21-17-39-21 +0000,nyt,Latest Trump-Zelensky Meeting Yields No Progre...,https://www.nytimes.com/2025/10/21/world/europ...,97
223,2025-10-21-21-57-30 +0000,bbc,Trump says he did not want 'wasted meeting' af...,https://www.bbc.com/news/articles/c4gjp73gp41o...,97
314,2025-10-21-17-10-44 +0000,nypost,Trump’s religious-freedom envoy pick visits Mi...,https://nypost.com/2025/10/21/world-news/trump...,96
301,2025-10-21-17-55-57 +0000,nypost,Hillary mocked after blasting Trump’s White Ho...,https://nypost.com/2025/10/21/us-news/hillary-...,96


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
63,104,2025-10-22-10-00-00 +0000,nypost,Sen. John Kennedy talks Trump’s social media h...,https://nypost.com/2025/10/22/us-news/sen-john...
166,76,2025-10-22-03-11-48 +0000,nyt,Vance Lands in Israel as U.S. Tries to Shore U...,https://www.nytimes.com/2025/10/21/world/middl...
201,63,2025-10-21-23-38-00 +0000,wsj,Paul Ingrassia withdrew Tuesday from a Senate ...,https://www.wsj.com/politics/policy/paul-ingra...
69,47,2025-10-22-10-00-00 +0000,nypost,Andrew Cuomo will try one last time to rattle ...,https://nypost.com/2025/10/22/us-news/andrew-c...
111,44,2025-10-22-09-00-00 +0000,wsj,The Republican National Committee heads into t...,https://www.wsj.com/politics/policy/democratic...
363,42,2025-10-21-13-56-46 +0000,nypost,Warner Bros. Discovery says it’s open to a sal...,https://nypost.com/2025/10/21/business/warner-...
302,42,2025-10-21-17-44-24 +0000,nypost,Hamas released bodies of two more dead hostage...,https://nypost.com/2025/10/21/world-news/hamas...
281,41,2025-10-21-19-05-13 +0000,nypost,Trump agencies ‘working to undo’ past abuses u...,https://nypost.com/2025/10/21/us-news/trump-ag...
212,37,2025-10-21-23-10-23 +0000,nypost,Netflix shares slide on rare earnings miss — s...,https://nypost.com/2025/10/21/media/netflix-sh...
256,37,2025-10-21-20-20-15 +0000,nypost,Here’s everything you need to know about Trump...,https://nypost.com/2025/10/21/business/heres-w...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
